In [1]:
%matplotlib inline

# Transfer Learning
In this assignment, we will use the weights of a network pre-trained in a particular problem as starting point to train our CNN to a different problem. As training a network from scratch is time-consuming and demands a lot of data, this is a frequent strategy, specially if both datasets (the one used for pre-training and the target) shares similar structures/elements/concepts. 

This is specially true when working with images. Most filters learned in initial convolutional layers will detect low-level elements, such as borders, corners and color blobs, which are common to most problems in the image domain. 

In this notebook, we will load the SqueezeNet architecture trained in the ImageNet dataset and fine-tune it to CIFAR-10.

## Imports

In [2]:
import os
import numpy as np
from random import sample, seed

from time import time
import keras.callbacks as callbacks

seed(42)
np.random.seed(42)

import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = (15,15) # Make the figures a bit bigger

# Keras imports
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, Flatten, Dense
from keras.models import Model
from keras import regularizers
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array
from keras.datasets import cifar10
from keras.callbacks import TensorBoard
from sklearn.cross_validation import StratifiedShuffleSplit
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

#Utility to plot
def plotImages(imgList):
    for i in range(len(imgList)):
        plotImage(imgList[i])
        
        
def plotImage(img):
    fig = plt.figure(figsize=(3,3))
    ax = fig.add_subplot(111)

    ax.imshow(np.uint8(img), interpolation='nearest')
    plt.show()

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## SqueezeNet definition
These methods define our architecture and load the weights obtained using ImageNet data.

In [3]:
# Fire Module Definition
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x

#SqueezeNet model definition
def SqueezeNet(input_shape):
    img_input = Input(shape=input_shape) #placeholder
    
    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = Dropout(0.5, name='drop9')(x)

    x = Convolution2D(1000, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax', name='loss')(x)

    model = Model(img_input, x, name='squeezenet')

    # Download and load ImageNet weights
    model.load_weights('./squeezenet_weights_tf_dim_ordering_tf_kernels.h5')
    
    return model    

## CIFAR-10

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. The class are **airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck**.

In [4]:
#Load data
(trainVal_data, trainVal_label), (X_test, y_test) = cifar10.load_data()
print("Train/Val data. X: ", trainVal_data.shape, ", Y: ", trainVal_data.shape)
print("Test data. X: ", X_test.shape, ", Y: ", y_test.shape)

# Prepare the data
# ...
train_x, val_x, train_labels, val_labels = train_test_split(trainVal_data, trainVal_label, test_size=0.2, random_state=0)


Train/Val data. X:  (50000, 32, 32, 3) , Y:  (50000, 32, 32, 3)
Test data. X:  (10000, 32, 32, 3) , Y:  (10000, 1)


In [5]:
from keras.utils.np_utils import to_categorical

train_categorical_labels = to_categorical(train_labels, num_classes=None)
val_categorical_labels = to_categorical(val_labels, num_classes=None)
test_categorical_labels = to_categorical(y_test, num_classes=None)

-----------------
## SqueezeNet with frozen layers
Our initial attempt will be to remove SqueezeNet's top layers --- responsible for the classification into ImageNet classes --- and train a new set of layers to our CIFAR-10 classes. We will also freeze the layers before `drop9`. Our architecture will be like this:

<img src="frozenSqueezeNet.png" width=70% height=70%>

In [6]:
from keras import models

squeezeNetModel = SqueezeNet((32,32,3))

#freeze layers
for layer in squeezeNetModel.layers:
    layer.trainable = False

#Add new classification layers
x = squeezeNetModel.layers[-5].output
x = Convolution2D(10, (1, 1), padding='valid', name='new_conv10')(x)
x = Activation('relu', name='new_relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='new_loss')(x)

#new Model
model = Model(squeezeNetModel.inputs, x, name='squeezenet_new')

Now, we compile our model and train it:

In [7]:
model.layers[-4].trainable

True

In [8]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=False)

datagen.fit(train_x)

In [9]:
from keras import optimizers

BATCHSIZE = 32

#Compile model
# ...
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr = 0.0001, momentum=0.9, nesterov=True), metrics=["accuracy"])

#Tensorboard callback
#tbCallBack = TensorBoard(log_dir="./logs/rafa", write_graph=True)
tbCallBacks = [callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3,
                                      verbose=0, mode='auto', baseline=None, 
                                      restore_best_weights=False),
              TensorBoard(log_dir="./monitor/1".format(time()), write_graph=True)]
    
#Train model
# ...

history = model.fit_generator(datagen.flow(train_x, train_categorical_labels, batch_size=BATCHSIZE),
                    validation_data=datagen.flow(val_x, val_categorical_labels, batch_size=BATCHSIZE),
                    validation_steps=len(val_x)/BATCHSIZE,
                    steps_per_epoch=len(train_x)/BATCHSIZE, 
                    epochs=100, use_multiprocessing=True,
                    initial_epoch=0,
                    callbacks=tbCallBacks)

Epoch 1/100
1250/1250 [==============================] - 12s 9ms/step - loss: 2.3217 - acc: 0.1207 - val_loss: 2.2565 - val_acc: 0.2040
Epoch 2/100
1250/1250 [==============================] - 11s 9ms/step - loss: 2.2511 - acc: 0.1792 - val_loss: 2.1974 - val_acc: 0.2538
Epoch 3/100
1250/1250 [==============================] - 11s 9ms/step - loss: 2.2089 - acc: 0.2162 - val_loss: 2.1481 - val_acc: 0.2752
Epoch 4/100
1250/1250 [==============================] - 11s 9ms/step - loss: 2.1755 - acc: 0.2383 - val_loss: 2.1130 - val_acc: 0.2868
Epoch 5/100
1250/1250 [==============================] - 12s 9ms/step - loss: 2.1494 - acc: 0.2481 - val_loss: 2.0872 - val_acc: 0.3003
Epoch 6/100
1250/1250 [==============================] - 12s 9ms/step - loss: 2.1220 - acc: 0.2626 - val_loss: 2.0573 - val_acc: 0.3121
Epoch 7/100
1250/1250 [==============================] - 12s 9ms/step - loss: 2.0879 - acc: 0.2699 - val_loss: 1.9848 - val_acc: 0.3272
Epoch 8/100
1250/1250 [=========================

Epoch 61/100
1250/1250 [==============================] - 11s 9ms/step - loss: 1.7753 - acc: 0.3662 - val_loss: 1.6680 - val_acc: 0.4234
Epoch 62/100
1250/1250 [==============================] - 11s 9ms/step - loss: 1.7751 - acc: 0.3676 - val_loss: 1.6688 - val_acc: 0.4213


Finally, let's evaluate on our test set:

In [12]:
# Evaluate on validation:
# ...
print(model.metrics_names)
print(model.evaluate_generator(datagen.flow(val_x, val_categorical_labels, batch_size=BATCHSIZE), steps=len(val_x)/BATCHSIZE))

['loss', 'acc']
[1.6702335821151733, 0.4243]


-----------------
-----------------

# Training last 2 Fire Modules + classification layers
As we could see, the frozen network performed very poorly. By freezing most layers, we do not allow SqueezeNet to adapt its weights to features present in CIFAR-10.

Let's try to unfreeze the last two fire modules and train once more. The architecture will be:
<img src="partFrozenSqueezeNet.png" width=70% height=70%>

In [17]:
squeezeNetModel = SqueezeNet((32,32,3))

#freeze the mentioned layers
# ...
for layer in squeezeNetModel.layers[:-19]:
    layer.trainable = False

#Add new classification layers
# ...
x = squeezeNetModel.layers[-5].output
x = Convolution2D(10, (1, 1), padding='valid', name='new_conv10')(x)
x = Activation('relu', name='new_relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='new_loss')(x)

#new Model
model = Model(squeezeNetModel.inputs, x, name='squeezenet_new')

In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 15, 15, 64)   1792        input_3[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 15, 15, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 7, 7, 64)     0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

In [19]:
model.layers[-20].trainable

False

Now, we compile our model and train it:

In [20]:
#Compile model and train it
# ...
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=False)

datagen.fit(train_x)

BATCHSIZE = 32

tbCallBacks = [callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3,
                                      verbose=0, mode='auto', baseline=None, 
                                      restore_best_weights=False),
              TensorBoard(log_dir="./monitor/2".format(time()), write_graph=True)]

#Compile model
# ...
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr = 0.0001, momentum=0.9, nesterov=True), metrics=["accuracy"])

#Train model
# ...

history = model.fit_generator(datagen.flow(train_x, train_categorical_labels, batch_size=BATCHSIZE),
                    validation_data=datagen.flow(val_x, val_categorical_labels, batch_size=BATCHSIZE),
                    validation_steps=len(val_x)/BATCHSIZE,
                    steps_per_epoch=len(train_x)/BATCHSIZE, 
                    epochs=100, use_multiprocessing=True,
                    initial_epoch=0,
                    callbacks=tbCallBacks)

Epoch 1/100
1250/1250 [==============================] - 15s 12ms/step - loss: 2.2418 - acc: 0.1782 - val_loss: 2.1159 - val_acc: 0.2698
Epoch 2/100
1250/1250 [==============================] - 14s 12ms/step - loss: 2.0911 - acc: 0.2592 - val_loss: 1.9646 - val_acc: 0.3393
Epoch 3/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.9207 - acc: 0.3218 - val_loss: 1.7347 - val_acc: 0.4030
Epoch 4/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.7778 - acc: 0.3603 - val_loss: 1.6350 - val_acc: 0.4200
Epoch 5/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.7081 - acc: 0.3830 - val_loss: 1.5868 - val_acc: 0.4387
Epoch 6/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.6621 - acc: 0.4025 - val_loss: 1.5483 - val_acc: 0.4472
Epoch 7/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.6287 - acc: 0.4166 - val_loss: 1.5282 - val_acc: 0.4532
Epoch 8/100
1250/1250 [==================

1250/1250 [==============================] - 14s 12ms/step - loss: 1.3688 - acc: 0.5153 - val_loss: 1.3320 - val_acc: 0.5307
Epoch 61/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.3630 - acc: 0.5164 - val_loss: 1.3190 - val_acc: 0.5341
Epoch 62/100
1250/1250 [==============================] - 14s 12ms/step - loss: 1.3678 - acc: 0.5179 - val_loss: 1.3148 - val_acc: 0.5374
Epoch 63/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.3609 - acc: 0.5166 - val_loss: 1.3231 - val_acc: 0.5341
Epoch 64/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.3630 - acc: 0.5164 - val_loss: 1.3134 - val_acc: 0.5383
Epoch 65/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.3582 - acc: 0.5186 - val_loss: 1.3079 - val_acc: 0.5366
Epoch 66/100
1250/1250 [==============================] - 14s 11ms/step - loss: 1.3623 - acc: 0.5185 - val_loss: 1.3112 - val_acc: 0.5386
Epoch 67/100
1250/1250 [=======================

Finally, let's evaluate on our test set:

In [21]:
# Evaluate on validation:
# ...
print(model.metrics_names)
print(model.evaluate_generator(datagen.flow(val_x, val_categorical_labels, batch_size=BATCHSIZE), steps=len(val_x)/BATCHSIZE))

['loss', 'acc']
[1.3119075336456298, 0.5364]


-----------
-----------
-----------
# Tensorboard

Tensorboard is a visualization tool for Tensorflow. Among other things, it allows us to monitor the progress of our training, plot metrics per epochs, visualize the architecture's schematics. 

Just like for Early Stopping, we will use the [Tensorboard callback](https://keras.io/callbacks/#tensorboard) to log the information about our training. An example of usage, would be:

As your training progresses, Keras will log the metrics (e.g., loss, accuracy) to `<<LOG_DIR>>` (**make sure `<<LOG_DIR>>` is a valid directory)**. On your terminal, you will need to run Tensorboard, assign a port and access it via browser (just like jupyter).

#### ----> MAKE SURE YOU USE A DIFFERENT PORT FOR JUPYTER AND TENSORBOARD <----

### Docker
For those using docker, open a new terminal and create a new container (using the same image) running Tensorboard:

For example:

After starting Tensorboard, access it via browser on `http://localhost:<<port_container>>`.

### Anaconda
$ tensorboard --logdir=<<LOG_DIR>> --port=<<port>>

After starting Tensorboard, access it via browser on `http://localhost:<<port>>`.

-----------
-----------
-----------

# Fine-tuning all layers

What if we fine-tune all layers of SqueezeNet?
<img src="unfrozenSqueezeNet.png" width=70% height=70%>

In [22]:
squeezeNetModel = SqueezeNet((32,32,3))

for layer in squeezeNetModel.layers:
    layer.trainable = True       #by default they are all trainable, but just for clarification

#Add new classification layers
# ...
x = squeezeNetModel.layers[-5].output
x = Convolution2D(10, (1, 1), padding='valid', name='new_conv10')(x)
x = Activation('relu', name='new_relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='new_loss')(x)

#new Model
model_adam = Model(squeezeNetModel.inputs, x, name='squeezenet_new_adam')

Now, we compile our model and train it:

In [ ]:
from time import time
import keras.callbacks as callbacks
BATCHSIZE = 32

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=False)

datagen.fit(train_x)

#Compile model
# ...
model_adam.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(), metrics=["accuracy"])

#Tensorboard callback

tbCallBacks = [callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3,
                                      verbose=0, mode='auto', baseline=None, 
                                      restore_best_weights=False),
              TensorBoard(log_dir="./monitor/3/adam".format(time()), write_graph=True)]

#Train model
# ...

history = model_adam.fit_generator(datagen.flow(train_x, train_categorical_labels, batch_size=BATCHSIZE),
                    validation_data=datagen.flow(val_x, val_categorical_labels, batch_size=BATCHSIZE),
                    validation_steps=len(val_x)/BATCHSIZE,
                    steps_per_epoch=len(train_x)/BATCHSIZE, 
                    epochs=100, use_multiprocessing=True,
                    initial_epoch=0,
                    callbacks=tbCallBacks)

Epoch 1/100
1250/1250 [==============================] - 30s 24ms/step - loss: 1.9475 - acc: 0.2585 - val_loss: 1.5548 - val_acc: 0.4341
Epoch 2/100
1250/1250 [==============================] - 29s 23ms/step - loss: 1.3286 - acc: 0.5306 - val_loss: 1.1291 - val_acc: 0.6053
Epoch 3/100
1250/1250 [==============================] - 28s 23ms/step - loss: 1.1056 - acc: 0.6197 - val_loss: 1.0364 - val_acc: 0.6522
Epoch 4/100
1250/1250 [==============================] - 29s 23ms/step - loss: 1.0030 - acc: 0.6621 - val_loss: 0.9705 - val_acc: 0.6728
Epoch 5/100
1250/1250 [==============================] - 29s 23ms/step - loss: 0.9429 - acc: 0.6848 - val_loss: 0.9234 - val_acc: 0.6957
Epoch 6/100
1250/1250 [==============================] - 30s 24ms/step - loss: 0.8914 - acc: 0.7050 - val_loss: 0.9056 - val_acc: 0.6971
Epoch 7/100
1250/1250 [==============================] - 29s 23ms/step - loss: 0.8472 - acc: 0.7200 - val_loss: 0.8743 - val_acc: 0.7071
Epoch 8/100
1250/1250 [==================

In [ ]:
squeezeNetModel = SqueezeNet((32,32,3))

for layer in squeezeNetModel.layers:
    layer.trainable = True       #by default they are all trainable, but just for clarification

#Add new classification layers
# ...
x = squeezeNetModel.layers[-5].output
x = Convolution2D(10, (1, 1), padding='valid', name='new_conv10')(x)
x = Activation('relu', name='new_relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='new_loss')(x)

#new Model
model_sgd = Model(squeezeNetModel.inputs, x, name='squeezenet_new_sgd')

In [ ]:
from time import time

BATCHSIZE=32

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=False)

datagen.fit(train_x)

#Compile model
# ...
model_sgd.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr = 0.0001, momentum=0.9, nesterov=True), metrics=["accuracy"])

#Tensorboard callback
tbCallBacks = [callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3,
                                      verbose=0, mode='auto', baseline=None, 
                                      restore_best_weights=False),
              TensorBoard(log_dir="./monitor/3/sgd".format(time()), write_graph=True)]

#Train model
# ...
history = model_sgd.fit_generator(datagen.flow(train_x, train_categorical_labels, batch_size=BATCHSIZE),
                    validation_data=datagen.flow(val_x, val_categorical_labels, batch_size=BATCHSIZE),
                    validation_steps=len(val_x)/BATCHSIZE,
                    steps_per_epoch=len(train_x)/BATCHSIZE, 
                    epochs=100, use_multiprocessing=True,
                    initial_epoch=0,
                    callbacks=tbCallBacks)

Finally, let's evaluate on our validation set:

In [ ]:
# Evaluate on validation
# ...

print(model_sgd.metrics_names)
print(model_sgd.evaluate_generator(datagen.flow(val_x, val_categorical_labels, batch_size=BATCHSIZE), steps=len(val_x)/BATCHSIZE))

print(model_adam.metrics_names)
print(model_adam.evaluate_generator(datagen.flow(val_x, val_categorical_labels, batch_size=BATCHSIZE), steps=len(val_x)/BATCHSIZE))

In [ ]:
# Evaluate your best model on test
# ...

## Saving the model
Now that we are working on more complex tasks and our trainings are starting to take more time it is usually a good idea to save the trained model from time to time. [Keras has a lot of ways of saving and loading the model](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model), but in this exercise we will use the simplest of them all: `model.save()`. It saves the architecture, the weights, the choice of loss function/optimizer/metrics and even the current state of the training, so you can resume your training later.

In [ ]:
model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'

## Loading a model
Once we have our model trained, we can load it using:

In [ ]:
from keras.models import load_model

del model  # Will delete model, only to check if load_model is working

# returns a compiled model identical to the previous one
model = load_model('my_model.h5')

# evaluate test set again... should give us the same result
# ...
print('Test loss:', score[0])
print('Test accuracy (NORMALIZED):', score[1])